In [1]:
import numpy as np 
import pandas as pd
import torch
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from pathlib import Path
from os.path import dirname, abspath
from tqdm import tqdm

In [2]:
#Altered to work on cluster ?
os.chdir("/datasets" + "/MaskedFace-Net")

In [3]:
transform_train = transforms.Compose(
    [transforms.Resize((100, 100)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [17]:
transform_val = transforms.Compose(
    [transforms.Resize((100, 100)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
transform_val = transforms.Compose(
    [transforms.CenterCrop(300),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
maskedface_net_train = torchvision.datasets.ImageFolder("../MaskedFace-Net/train", transform=transform_train)

In [18]:
maskedface_net_val = torchvision.datasets.ImageFolder("../MaskedFace-Net/validation", transform=transform_val)

In [7]:
maskedface_net_test = torchvision.datasets.ImageFolder("../MaskedFace-Net/holdout", transform=transform_val)

In [33]:
os.chdir('/datasets/home/19/219/chl820/DSC180B-Face-Mask-Detection')

In [8]:
data_loader_train = torch.utils.data.DataLoader(maskedface_net_train,
                                          batch_size=32, #4
                                          shuffle=True)

In [19]:
data_loader_val = torch.utils.data.DataLoader(maskedface_net_val,
                                             batch_size=32, #4
                                             shuffle=True)

In [10]:
data_loader_test = torch.utils.data.DataLoader(maskedface_net_test,
                                             batch_size=32, #4
                                             shuffle=True)

In [11]:
maskedface_net_train

#Incorrect provides us the labels, Example: ###_Mask_Mouth_Chin means Mask is present, Mouth and Chin Covered should be labeled
#as [1, 1, 1, 0] for Mask, Mouth, Chin, Nose
#Notes: [1, 1, 1, 1] suggest correct wear, [1, 1, 1, 0] suggests cover nose!, [0, 0, 0, 0] suggest no mask

#Currently 0 is Correct, 1 No Mask, 2 Wrong for labels

Dataset ImageFolder
    Number of datapoints: 37500
    Root location: ../MaskedFace-Net/train
    StandardTransform
Transform: Compose(
               Resize(size=(100, 100), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 3 input image channel, 1 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(3, 1, 3)
        self.conv2 = nn.Conv2d(1, 1, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(529, 120) 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #print(x.shape)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
%%time
#Takes a while, maybe few hours??

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(tqdm(data_loader_train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
            
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

  9%|▊         | 100/1172 [01:46<19:00,  1.06s/it]

[1,   100] loss: 2.204


 17%|█▋        | 200/1172 [03:33<16:58,  1.05s/it]

[1,   200] loss: 1.935


 26%|██▌       | 300/1172 [05:22<15:56,  1.10s/it]

...

[2,  1000] loss: 0.073


 94%|█████████▍| 1100/1172 [18:17<01:09,  1.03it/s]

[2,  1100] loss: 0.055


100%|██████████| 1172/1172 [19:28<00:00,  1.00it/s]

Finished Training
CPU times: user 57min 1s, sys: 1min 14s, total: 58min 15s
Wall time: 40min 41s


In [34]:
torch.save(net.state_dict(), 'models/model_v1.pt')

In [27]:
#Accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in tqdm(data_loader_val):
        inputs, labels = data
        outputs = net(inputs)
        predicted = labels
        total += labels.size(0)
        label_tensor_maxs = torch.tensor([torch.argmax(x) for x in labels])
        correct += (predicted == label_tensor_maxs).sum().item()

print('Accuracy of the network on the test set: %d %%' % (100 * correct / total))

  1%|          | 3/293 [00:03<05:10,  1.07s/it]


KeyboardInterrupt: 